## GOAL: Predict violation type via violation type code using other features

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('./data/data-1552240551240.csv')

C:\Users\justi\Anaconda3\envs\deeplearning\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,bbl,isndobbisviol,boro,bin,block,lot,issuedate,violationtypecode,violationnumber,housenumber,street,dispositiondate,dispositioncomments,devicenumber,description,ecbnumber,number,violationcategory,violationtype
0,3.014790e+09,1018012,3,3039615.0,01479,00001,2005-01-31,LL6291,15504,880,QUINCY STREET,2005-07-21,IRICIA DISMISSED FULL PENALTY PAID $500 ...,00081537,NaN,NaN,V*013105LL629115504,V*-DOB VIOLATION - DISMISSED,LL6291-LOCAL LAW 62/91 - BOILERS
1,1.014350e+09,798462,1,1044200.0,01435,00016,2002-03-26,LL6291,04391,330,EAST 61 STREET,2006-06-23,CIAMGZ CANCELLED DEED SUB NEW OWNER AS OF ...,00111092,NaN,NaN,V*032602LL629104391,V*-DOB VIOLATION - DISMISSED,LL6291-LOCAL LAW 62/91 - BOILERS
2,1.016710e+09,251606,1,1052648.0,01671,00049,1993-03-18,LL6291,10195,1936,2 AVENUE,2000-08-08,"VTUCIA CANCELLED, BLDG HEATED BY 1932 ...",00900563,NaN,NaN,V*031893LL629110195,V*-DOB VIOLATION - DISMISSED,LL6291-LOCAL LAW 62/91 - BOILERS
3,2.025230e+09,1071206,2,2097167.0,02523,00061,2006-02-02,E,9444/153715,950,UNIVERSITY AVEN,2007-06-11,PPN203 AOC SUBMITTED ON 06/08/2007 BY DYNAMIC ...,002P1188,NaN,NaN,V*020206E9444/153715,V*-DOB VIOLATION - Resolved,E-ELEVATOR
4,2.031440e+09,2027969,2,2013386.0,03144,00066,2016-02-05,IMEGNCY,9246,2094,VALENTINE AVENUE,2016-02-11,BLDING IS NOT DEMOLISHED. FENCE ERECTED IN PLA...,NaN,02/05/16 DOB : DEC 9246 HAS BEEN UPLOADED AND ...,NaN,V*020516IMEGNCY9246,V*-DOB VIOLATION - DISMISSED,IMEGNCY-IMMEDIATE EMERGENCY


In [5]:
df.columns

Index(['bbl', 'isndobbisviol', 'boro', 'bin', 'block', 'lot', 'issuedate',
       'violationtypecode', 'violationnumber', 'housenumber', 'street',
       'dispositiondate', 'dispositioncomments', 'devicenumber', 'description',
       'ecbnumber', 'number', 'violationcategory', 'violationtype'],
      dtype='object')

In [10]:
for column in df.columns:
    print(column, ' ', df[column].nunique())

bbl   215454
isndobbisviol   2020288
boro   17
bin   218799
block   24574
lot   2414
issuedate   12654
violationtypecode   52
violationnumber   799405
housenumber   45077
street   15766
dispositiondate   10078
dispositioncomments   466754
devicenumber   353184
description   100615
ecbnumber   146632
number   2013506
violationcategory   14
violationtype   46


y = violationcategory (as it reflects violationtype; so drop violationtype column altogether)

1. trying categorical approach first
    - problem: most features seem non-categorical
2. try neural network with concatenated features
    - remove irrelevant features like violation number first
    - perform concatenation
    - train NN